In [1]:
import librosa  # for audio processing
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile  # for audio processing
from numpy.lib.stride_tricks import as_strided
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import Audio
import sklearn
import pandas as pd
import json
import os
from os.path import exists
import warnings
warnings.filterwarnings("ignore")
# from cleaner import CleanDataFrame

In [2]:
# %cd drive/MyDrive/Week-4-STT/data/AMHARIC/data/
%cd ../data/

c:\Users\User\Desktop\10AcademyProjects\speech-to-text\Speech-to-Text\data


In [3]:
train_dataset_location = 'train/'
train_wav_location = os.path.join(train_dataset_location, 'wav/')
train_changed_wav_location = os.path.join(train_dataset_location, 'changed_wav/')
train_txt_location = os.path.join(train_dataset_location, 'trsTrain.txt')
test_dataset_location = 'test/' 
lexicon_location = 'train/'

In [4]:
def convert_to_sterio(audio: np.array) -> np.array:
        if len(audio.shape) == 1:
            sterio = np.stack([audio, audio], axis=1)
            return sterio
        return audio


from scipy.io import wavfile

def resize_audio(audio: np.array, size: int) -> np.array:
        """
        This resizes all input audio to a fixed sample size.
        It helps us to have a consistent data shape

        Args:
            audio: This is the audio sample as a numpy array
        """
        resized = librosa.util.fix_length(audio, size, axis=0)
        print(f"Audio resized to {size} samples")
        return resized
# changed = convert_to_sterio(samples)
# changed.shape  
# resized = resize_audio(changed,200000)
# print(resized.shape)
# print(resized.T.shape)
# wavfile.write(os.path.join(train_changed_wav_location, 'trial.wav'), sample_rate, resized)
# ipd.Audio(resized.T, rate=sample_rate)
# # mfcc = librosa.feature.mfcc(y=changed, sr=sample_rate)
# # mfcc.shape
# samples, sample_rate = librosa.load(train_changed_wav_location+'trial.wav' , sr=44100)
# samples.shape
# wav_roll = np.roll(samples, int(sample_rate/10))

def meta_data(trans, path):
        target = []
        features = []
        mode = []
        rmse = []
        spec_cent = []
        spec_bw = []
        rolloff = []
        zcr = []
        mfcc = []
        rate = []
        filenames = []
        duration_of_recordings = []
        for index, k in enumerate(trans):
            if index < 5:
                filename = path + k + ".wav"
                next_file_name = path + k + "changed.wav"
                if exists(filename):
                    audio, fs = librosa.load(filename, sr=44100)
                   
                    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=fs)
                    rmse.append(np.mean(librosa.feature.rms(y=audio)))
                    spec_cent.append(
                        np.mean(librosa.feature.spectral_centroid(y=audio, sr=fs)))
                    spec_bw.append(
                        np.mean(librosa.feature.spectral_bandwidth(y=audio, sr=fs)))
                    rolloff.append(
                        np.mean(librosa.feature.spectral_rolloff(y=audio, sr=fs)))
                    zcr.append(
                        np.mean(librosa.feature.zero_crossing_rate(audio)))
                    mfcc.append(np.mean(librosa.feature.mfcc(y=audio, sr=fs)))
                    duration_of_recordings.append(float(len(audio)/fs))
                    rate.append(fs)
                    changed = convert_to_sterio(audio)
                    audio = resize_audio(changed,200000)
                   
                    # stereo = change_channel_to_stereo(filename, next_file_name)
                    # resized = self.resize_audio(audio,200000)
                    split_array = str(filename).split('/')
                    filename = '../data/train/changed_wav/' + str(split_array[len(split_array)-1])
                    wavfile.write(os.path.join(train_changed_wav_location, str(split_array[len(split_array)-1]) ), fs, audio)
                    filenames.append(filename)
                    mode.append('mono')  # if stereo == 1 else 'stereo')
                    lable = trans[k]
                    target.append(lable)
        # self.logger.info(f"Meta Data Generated For {len(filenames)} Audios")
        return filenames, target, duration_of_recordings, mode, rmse, spec_cent, spec_bw, rolloff, zcr, mfcc, rate

In [5]:
def loaderTrans(filename: str):
        """
        # Loads the audio file and returns the audio data and sample rate
        # @param filename: The path to the audio file
        # @return: The audio data and sample rate
        #
        """
        name_to_text = {}
        with open(filename, encoding="utf-8") as f:
            for line in f:
                name = line.split("</s>")[1]
                name = name.replace("(", "")
                name = name.replace(")", "")
                name = name.replace("\n", "")
                name = name.replace(" ", "")
                text = line.split("</s>")[0]
                text = text.replace("<s>", "")
                name_to_text[name] = text
                # self.logger.info(f"Training data loaded: {name}")
        return name_to_text
transcription = loaderTrans(train_txt_location)

In [6]:
filenames, target, duration_of_recordings,mode ,rmse,spec_cent,spec_bw,rolloff,zcr,mfcc,rate = meta_data(transcription, train_wav_location)
data = pd.DataFrame({'key': filenames, 'text': target,
                    'duration': duration_of_recordings, 'mode': mode , 'rate': rate ,'rmse': rmse,'spec_cent' :spec_cent,'spec_bw': spec_bw,"rolloff" :rolloff,"zcr": zcr,"mfcc": mfcc})
data.head()

Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples


,key,text,duration,mode,rate,rmse,spec_cent,spec_bw,rolloff,zcr,mfcc
0,../data/train/changed_wav/tr_1_tr01001.wav,ያንደኛ ደረጃ ትምህርታቸው ን ጐንደር ተ ም ረዋል,4.608005,mono,44100,0.005627,713.781516,1236.804586,1485.248618,0.004273,-18.103716
1,../data/train/changed_wav/tr_2_tr01002.wav,የተ ለቀቁት ምርኮኞች በ አካባቢያቸው ሰላማዊ ኑሮ እንዲ ኖሩ የ ትራንስ...,16.384014,mono,44100,0.005557,769.171143,1256.989803,1618.360516,0.003095,-17.840212
2,../data/train/changed_wav/tr_3_tr01003.wav,በ አዲስ አበባው ስታዲየም በ ተካሄዱ ት ሁለት ግጥሚያ ዎች በ መጀመሪያ...,14.592018,mono,44100,0.005417,783.725021,1286.675508,1654.442077,0.001926,-18.258934
3,../data/train/changed_wav/tr_4_tr01004.wav,ወሬው ን ወሬ ያደረጉ ምስጢረ ኞች ናቸው,4.736009,mono,44100,0.005535,687.372255,1161.387180,1368.097283,0.004953,-18.796532
4,../data/train/changed_wav/tr_5_tr01005.wav,ኢትዮጵያዊ ቷ በ ብሄራዊ ባህላዊ አለባበስ ከ አለም አንደኝነት ን ተቀዳ...,8.192018,mono,44100,0.005626,782.274709,1257.456211,1691.149453,0.003393,-18.172558


In [7]:
train_json = data[:8000].to_dict(orient='records')
with open("train_corpus.json", "w", encoding='UTF-8') as export_file:
            json.dump(train_json, export_file, indent=4,
                     sort_keys=True, ensure_ascii=False)
valid_json = data[8001:].to_dict(orient='records')
with open("valid_corpus.json", "w", encoding='UTF-8') as export_file:
            json.dump(valid_json, export_file, indent=4,
                     sort_keys=True, ensure_ascii=False)